### **Comparison of Random Number Generators**

In [1]:
"""
1. Consider a six months European put option with strike 55 on a non–dividend–paying
underlying asset with spot price 50 following a lognormal distribution with volatility 30%.
Assume that the risk-free rate is constant at 4%.
Compute the Black–Scholes value V of the option.
"""
import numpy as np
from scipy.stats import norm
import pandas as pd

# option parameters
K = 55
S0 = 50
sigma = 0.3
r = 0.04
T = 0.5
q = 0

def bs_put(S0, K, T, r, sigma):
    """
    Black-Scholes formula for European put option
    """
    d1 = (np.log(S0 / K) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return K * np.exp(-r * T) * norm.cdf(-d2) - S0 * np.exp(-q * T) * norm.cdf(-d1)

print("Black-Scholes value of the option: ", bs_put(S0, K, T, r, sigma))


Black-Scholes value of the option:  6.6166546586411705


**Inverse transform**

In [16]:
"""
Find an approximate option values using Monte Carlo simulations.

Let
Si = S(0)*exp((r-q-1/2*sigma^2)*T + sigma*sqrt(T)*zi), i = 1 : M,
Vi = e^(-rT)*max(K-Si, 0), i = 1 : M,
where zi, i = 1 : M, are independent samples of the standard normal variable obtained from N uniform samples.
Note that number of samples M = N_{A-R} and M = N_{B-M} corresponding to the Acceptance–
Rejection method and to the Marsaglia–Bray version of the Box–Muller method are smaller
than N. Report the approximate values \hat{V}(n) = \sum_{i=1}^n V_i/n and the corresponding approximation errors
|VBS - Vb(n)| in the tables below.

"""

# inverse transform
def inverse_transform(N, T, r, S0, sigma, K, q):
    """
    Inverse transform method
    """
    np.random.seed(42)
    U = np.random.uniform(0, 1, N)
    Z = norm.ppf(U)
    V = S0 * np.exp((r - 0.5 * sigma ** 2) * T + sigma * np.sqrt(T) * Z)

    # calculate payoff
    P = np.exp(-r*T) * (K-V)
    P[P < 0] = 0

    return P.mean(), np.abs(bs_put(S0, K, T, r, sigma) - P.mean())

K = 55
S0 = 50
sigma = 0.3
r = 0.04
T = 0.5
q = 0

nums = [10000, 20000, 40000, 80000, 160000, 320000, 640000, 1280000, 2560000, 5120000]
res = pd.DataFrame(columns=['N', 'V', 'error'])
for n in nums:
    res.loc[len(res)] = [n] + list(inverse_transform(n, T, r, S0, sigma, K, q))
res

,N,V,error
0,10000.0,6.730943,0.114289
1,20000.0,6.630084,0.013429
2,40000.0,6.621701,0.005046
3,80000.0,6.608744,0.007911
4,160000.0,6.608961,0.007694
5,320000.0,6.608559,0.008096
6,640000.0,6.612866,0.003789
7,1280000.0,6.612947,0.003707
8,2560000.0,6.617005,0.000351
9,5120000.0,6.616569,0.000086


**Acceptance Rejection Method**

In [27]:
%%time

# acceptance-rejection
def acceptance_rejection(N, T, r, S0, sigma, K, q):
    """
    Acceptance-rejection method
    """
    np.random.seed(42)
    Z = []
    for _ in range(N):
        while True:
            u1, u2, u3 = np.random.rand(3)
            X = -np.log(u1)
            if u2 > np.exp(-(X-1)**2):
                continue
            else:
                if u3 <= 0.5:
                    X = -X
                Z.append(X)
                break
    Z = np.array(Z)
    V = S0 * np.exp((r - 0.5 * sigma ** 2) * T + sigma * np.sqrt(T) * Z)
    
    # calculate payoff
    P = np.exp(-r*T) * (K-V)
    P[P < 0] = 0

    return P.mean(), np.abs(bs_put(S0, K, T, r, sigma) - P.mean())   

K = 55
S0 = 50
sigma = 0.3
r = 0.04
T = 0.5
q = 0

nums = [10000, 20000, 40000, 80000, 160000, 320000, 640000, 1280000, 2560000, 5120000]
res = pd.DataFrame(columns=['N', 'V', 'error'])
for n in nums:
    res.loc[len(res)] = [n] + list(acceptance_rejection(n, T, r, S0, sigma, K, q))
res

,N,V,error
0,10000.0,6.540646,0.076009
1,20000.0,6.475726,0.140928
2,40000.0,6.507676,0.108979
3,80000.0,6.514461,0.102194
4,160000.0,6.534600,0.082055
5,320000.0,6.542623,0.074031
6,640000.0,6.536164,0.080491
7,1280000.0,6.533418,0.083237
8,2560000.0,6.533740,0.082915
9,5120000.0,6.535408,0.081247


**Box-Muller Method**

In [29]:
%%time

# Marsaglia-Bray
def marsaglia_bray(N, T, r, s0, sigma, K, q):
    """
    Marsaglia-Bray method
    """
    np.random.seed(42)
    Z = []
    for _ in range(N):
        X = 2
        while X > 1:
            u1, u2 = np.random.rand(2)
            u1 = 2 * u1 - 1
            u2 = 2 * u2 - 1
            X = u1**2 + u2**2
        Y = np.sqrt(-2 * np.log(X) / X)
        Z1 = u1 * Y
        Z2 = u2 * Y
        Z.extend([Z1, Z2])
    Z = np.array(Z)
    V = S0 * np.exp((r - 0.5 * sigma ** 2) * T + sigma * np.sqrt(T) * Z)
    
    # calculate payoff
    P = np.exp(-r*T) * (K-V)
    P[P < 0] = 0

    return P.mean(), np.abs(bs_put(S0, K, T, r, sigma) - P.mean())   

K = 55
S0 = 50
sigma = 0.3
r = 0.04
T = 0.5
q = 0

nums = [10000, 20000, 40000, 80000, 160000, 320000, 640000, 1280000, 2560000, 5120000]
res = pd.DataFrame(columns=['N', 'V', 'error'])
for n in nums:
    res.loc[len(res)] = [n] + list(marsaglia_bray(n, T, r, S0, sigma, K, q))
res

Wall time: 1min 3s


,N,V,error
0,10000.0,6.586618,0.030037
1,20000.0,6.634818,0.018163
2,40000.0,6.613519,0.003135
3,80000.0,6.606304,0.010351
4,160000.0,6.621881,0.005227
5,320000.0,6.627266,0.010612
6,640000.0,6.628741,0.012086
7,1280000.0,6.620805,0.004150
8,2560000.0,6.617689,0.001035
9,5120000.0,6.617765,0.001111
